In [68]:
import os
NN_ROOT = os.environ['NN_ROOT']
import sys
sys.path.append(NN_ROOT)
import numpy as np
from easydict import EasyDict
import torch
import torch.nn as nn
# import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
import shutil
import copy
import json
from torch.utils.data import TensorDataset, DataLoader
from pertnet.data.data_utils import load_data, save_data
from pertnet.net.pertnet_utils import (plot_response_coeffs, make_shot_response_movie, 
                            plot_loss_curve, train, MLP, DataPreProcess, visualize_response_prediction, 
                            plot_response_timetraces)

print('Loading parameters...')

# load parameters
fn = os.getcwd() + '/args.json'
with open(fn) as infile:
    hp = EasyDict(json.load(infile))

hp.traindata_fn = 'pertnet/data/datasets/train_013.dat'
hp.valdata_fn = 'pertnet/data/datasets/val_013.dat'
hp.testdata_fn = 'pertnet/data/datasets/test_013.dat'
    

# load data
print('Loading data...')
traindata = load_data(hp.root + hp.traindata_fn)
valdata = load_data(hp.root + hp.valdata_fn)
testdata = load_data(hp.root + hp.testdata_fn)


# process dataset (normalize, randomize, etc)
print('Normalizing data...')
preprocess = DataPreProcess(traindata, hp.xnames, hp.ynames, t_thresh=None)
trainX, trainY,_,_ = preprocess.transform(traindata, randomize=True, holdback_fraction=0.1)
valX, valY,_,_ = preprocess.transform(valdata, randomize=True, holdback_fraction=0.0)


Loading parameters...
Loading data...


/Users/jwai/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PCA from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Normalizing data...


In [69]:
keys = list(testdata.keys())
data_pca = copy.deepcopy(traindata)
keys.remove('shot')
keys.remove('time')


shots = np.vstack([traindata['shot'], valdata['shot'], testdata['shot']])
time = np.vstack([traindata['time'], valdata['time'], testdata['time']])

shotlist = np.sort(np.unique(shots))

isort = np.array([])

for shot in shotlist:
    i = np.where(shots==shot)[0]
    i = np.sort(i)
    isort = np.hstack([isort, i])
    
isort = isort.astype(int)
data_pca['shot'] = shots[isort]
data_pca['time'] = time[isort]


for key in keys:
    coeff_ = np.vstack([traindata[key].coeff_, valdata[key].coeff_, testdata[key].coeff_])
    data_pca[key].coeff_ = coeff_[isort,:]
    

In [70]:
fn = NN_ROOT + 'pertnet/data/datasets/data_pca_013.dat'
save_data(data_pca, fn)